<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamB/blob/main/Datawarehouse_TeamB_V1_20240204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import sqlite3
from sqlite3 import Error

# Define the file paths for source CSV files



In [6]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
patients_csv = '/content/drive/MyDrive/patients.csv'
medications_csv = '/content/drive/MyDrive/medications.csv'
conditions_csv = '/content/drive/MyDrive/conditions.csv'
observations_csv = '/content/drive/MyDrive/observations.csv'

# Create a SQLite database connection

In [39]:
db_path = '/content/drive/MyDrive/TeamB_Warehouse.db'
conn = sqlite3.connect(db_path)



# Read CSV files into pandas DataFrames

In [40]:
patients_df = pd.read_csv(patients_csv)
medications_df = pd.read_csv(medications_csv)
conditions_df = pd.read_csv(conditions_csv)
observations_df = pd.read_csv(observations_csv)



# Create tables in the SQLite database

In [41]:
patients_df.to_sql('patients', conn, index=False, if_exists='replace')
medications_df.to_sql('medications', conn, index=False, if_exists='replace')
conditions_df.to_sql('conditions', conn, index=False, if_exists='replace')
observations_df.to_sql('observations', conn, index=False, if_exists='replace')



31555

# Execute SQL commands to create and populate the target table

In [43]:
sql_commands = """
-- Leere FACT-Tabelle erstellen
DROP TABLE IF EXISTS F_BreastCancerQoL;

CREATE TABLE F_BreastCancerQoL (
  PatientId UUID,
  Age_group_diagnosis_ID Integer,
  Comorbidity_ID Integer,
  Comorbidity_YesNo Integer,
  Ethnicity Integer,
  Gender String,
  Cancertherapy_ID Integer,
  Chemotherapy_YesNo Integer,
  Targetedtherapy_YesNo Integer,
  Medication_ID Integer,
  Medication_YesNo Integer,
  QoL_Value_first Integer,
  QoL_Value_last Integer
);

-- Temporäre Tabelle mit MedicationID (=Medication.Code) erstellen (ohne Krebstherapie):
DROP TABLE IF EXISTS temp_patients_medicationID;

CREATE TABLE temp_patients_medicationID AS
SELECT DISTINCT p.id, m.code
FROM patients p
JOIN medications m ON p.id = m.patient
JOIN conditions c ON p.id = c.patient
WHERE m.reasoncode != '254837009' AND c.code = '254837009';

-- Temporäre Tabelle mit CancertherapieID (= Medication.Code):
DROP TABLE IF EXISTS temp_patients_cancertherapyID;

CREATE TABLE temp_patients_cancertherapyID AS
SELECT DISTINCT p.id, m.code
FROM patients p
JOIN medications m ON p.id = m.patient
JOIN conditions c ON p.id = c.patient
WHERE m.reasoncode = '254837009' AND c.code = '254837009';

-- Temporäre Tabelle mit PatientID erstellen (Nur Brustkrebspatienten):
DROP TABLE IF EXISTS temp_patients;

CREATE TABLE temp_patients AS
SELECT DISTINCT p.id, p.ethnicity, p.gender
FROM patients p
JOIN observations o ON p.id = o.patient
JOIN conditions c ON p.id = c.patient
WHERE c.code = '254837009';

-- Temporäre Tabelle mit Comorbidities erstellen (ohne Brustkrebserkrankungen):
DROP TABLE IF EXISTS temp_patients_comorbidity;

CREATE TABLE temp_patients_comorbidity AS
SELECT DISTINCT p.id, c.code
FROM conditions c
JOIN temp_patients p ON p.id = c.patient
WHERE c.code != '254837009';

-- Temporäre Tabelle mit erstem QoL Wert erstellen
DROP TABLE IF EXISTS temp_qol_value_first;

CREATE TABLE temp_qol_value_first AS
SELECT o.value AS qol_value_first, c.patient, MIN(o.date) AS earliest_date
FROM observations o
JOIN conditions c ON o.PATIENT = c.patient
WHERE o.code = 'QOLS' AND c.code = '254837009'
GROUP BY c.patient
ORDER BY c.patient;

-- Temporäre Tabelle mit letztem QoL Wert erstellen:
DROP TABLE IF EXISTS temp_qol_value_last;

CREATE TABLE temp_qol_value_last AS
SELECT o.value AS qol_value_last, c.patient, MAX(o.date) AS latest_date
FROM observations o
JOIN conditions c ON o.PATIENT = c.patient
WHERE o.code = 'QOLS' AND c.code = '254837009'
GROUP BY c.patient
ORDER BY c.patient;

-- d_Patients Tabelle erstellen:
DROP TABLE IF EXISTS d_Patients;

CREATE TABLE d_Patients AS
SELECT
  p.id AS Id,
  STRFTIME('%Y', 'now') - STRFTIME('%Y', p.birthdate) - (STRFTIME('%m-%d', 'now') < STRFTIME('%m-%d', p.birthdate)) AS AGE_Studyinclusion,
  STRFTIME('%Y', c.START) - STRFTIME('%Y', p.birthdate) - (STRFTIME('%m-%d', c.START) < STRFTIME('%m-%d', p.birthdate)) AS AGE_diagnosis,
  p.marital AS Maritalstatus
FROM
  patients p
JOIN conditions c ON p.id = c.patient
WHERE c.code = '254837009';

-- d_Medications Tabelle erstellen:
DROP TABLE IF EXISTS d_Medications;

CREATE TABLE d_Medications AS
SELECT DISTINCT
  code AS ID,
  CASE
    WHEN Description LIKE '%Clopidogrel%' THEN 'Clopidogrel'
    WHEN Description LIKE '%Simvastatin%' THEN 'Simvastatin'
    WHEN Description LIKE '%Amlodipine%' THEN 'Amlodipine'
    WHEN Description LIKE '%Nitroglycerin%' THEN 'Nitroglycerin'
    WHEN Description LIKE '%Warfarin%' THEN 'Warfarin'
    WHEN Description LIKE '%Verapamil%' THEN 'Verapamil'
    WHEN Description LIKE '%Digoxin%' THEN 'Digoxin'
    WHEN Description LIKE '%Epinephrine%' THEN 'Epinephrine'
    WHEN Description LIKE '%Amiodarone%' THEN 'Amiodarone'
    WHEN Description LIKE '%Alteplase%' THEN 'Alteplase'
    WHEN Description LIKE '%Atropine%' THEN 'Atropine'
    WHEN Description LIKE '%Atorvastatin%' THEN 'Atorvastatin'
    WHEN Description LIKE '%Captopril%' THEN 'Captopril'
  END AS Description
FROM medications
WHERE Description LIKE '%Clopidogrel%'
   OR Description LIKE '%Simvastatin%'
   OR Description LIKE '%Amlodipine%'
   OR Description LIKE '%Nitroglycerin%'
   OR Description LIKE '%Warfarin%'
   OR Description LIKE '%Verapamil%'
   OR Description LIKE '%Digoxin%'
   OR Description LIKE '%Epinephrine%'
   OR Description LIKE '%Amiodarone%'
   OR Description LIKE '%Alteplase%'
   OR Description LIKE '%Atropine%'
   OR Description LIKE '%Atorvastatin%'
   OR Description LIKE '%Captopril%';

-- d_Cancertherapy Tabelle erstellen:
DROP TABLE IF EXISTS d_Cancertherapy;

CREATE TABLE d_Cancertherapy AS
SELECT DISTINCT
  code AS ID,
  CASE
    WHEN Description LIKE '%Anastrozole%' THEN 'Anastrozole (AI)'
    WHEN Description LIKE '%Trastuzumab%' THEN 'Trastuzumab (anti-HER2 antibody)'
    WHEN Description LIKE '%Palbociclib%' THEN 'Palbociclib (CDK4/6 inhibitor)'
    WHEN Description LIKE '%Tamoxifen%' THEN 'Tamoxifen (SERM)'
    WHEN Description LIKE '%Verzenio%' THEN 'Abemaciclib (CDK4/6 inhibitor)'
    WHEN Description LIKE '%Epirubicin%' THEN 'Epirubicin'
    WHEN Description LIKE '%Cyclophosphamide%' THEN 'Cyclophosphamide'
    WHEN Description LIKE '%Paclitaxel%' THEN 'Paclitaxel'
  END AS Description,
  CASE
    WHEN Description LIKE '%Epirubicin%' THEN 1
    WHEN Description LIKE '%Cyclophosphamide%' THEN 1
    WHEN Description LIKE '%Paclitaxel%' THEN 1
    ELSE 0
  END AS Chemotherapy,
  CASE
    WHEN Description LIKE '%Anastrozole%' THEN 1
    WHEN Description LIKE '%Trastuzumab%' THEN 1
    WHEN Description LIKE '%Palbociclib%' THEN 1
    WHEN Description LIKE '%Tamoxifen%' THEN 1
    WHEN Description LIKE '%Verzenio%' THEN 1
    ELSE 0
  END AS TargetedTherapy
FROM medications
WHERE Description LIKE '%Anastrozole%'
   OR Description LIKE '%Trastuzumab%'
   OR Description LIKE '%Palbociclib%'
   OR Description LIKE '%Tamoxifen%'
   OR Description LIKE '%Verzenio%'
   OR Description LIKE '%Epirubicin%'
   OR Description LIKE '%Cyclophosphamide%'
   OR Description LIKE '%Paclitaxel%';

--d_Comorbidity erstellen:
DROP TABLE IF EXISTS d_Comorbidity;

CREATE TABLE d_Comorbidity AS
SELECT DISTINCT code AS ID, description AS Description
FROM conditions
WHERE code != '254837009';

-- Fülle F_BreastCancerQoL auf:
INSERT INTO F_BreastCancerQoL (PatientId, Age_group_diagnosis_ID, Comorbidity_ID, Comorbidity_YesNo, Ethnicity, Gender, Cancertherapy_ID, Chemotherapy_YesNo, Targetedtherapy_YesNo, Medication_ID, Medication_YesNo, QoL_Value_first, QoL_Value_last)
SELECT
  p.id AS PatientId,
  CASE
    WHEN d.Age_diagnosis < 70 THEN 1
    WHEN d.age_diagnosis >= 70 THEN 2
    ELSE NULL
  END AS Age_group_diagnosis_ID,
  c.code AS Comorbidity_ID,
  CASE WHEN c.code IS NOT NULL THEN 1 ELSE 0 END AS Comorbidity_YesNo,
  p.ethnicity AS Ethnicity,
  p.gender AS Gender,
  ct.code AS Cancertherapy_ID,
  dct.chemotherapy AS Chemotherapy_YesNo,
  dct.targetedtherapy AS Targetedtherapy_YesNo,
  m.code AS Medication_ID,
  CASE WHEN m.code IS NOT NULL THEN 1 ELSE 0 END AS Medication_YesNo,
  qvf.qol_value_first AS QoL_Value_first,
  qvl.qol_value_last AS QoL_Value_last
FROM d_patients d
LEFT JOIN temp_patients_comorbidity c ON d.id = c.id
LEFT JOIN temp_patients_cancertherapyID ct ON d.id = ct.id
LEFT JOIN temp_patients_medicationID m ON d.id = m.id
LEFT JOIN temp_qol_value_first qvf ON d.id = qvf.patient
LEFT JOIN temp_qol_value_last qvl ON d.id = qvl.patient
LEFT JOIN temp_patients p ON d.id = p.id
JOIN d_Cancertherapy dct ON ct.code = dct.ID;

-- Drop temp tables

DROP TABLE IF EXISTS temp_patients;
DROP TABLE IF EXISTS temp_patients_cancertherapyID;
DROP TABLE IF EXISTS temp_patients_medicationID;
DROP TABLE IF EXISTS temp_patients_comorbidity;
DROP TABLE IF EXISTS temp_qol_value_first;
DROP TABLE IF EXISTS temp_qol_value_last;

-- Drop other tables

DROP TABLE IF EXISTS medications;
DROP TABLE IF EXISTS conditions;
DROP TABLE IF EXISTS observations;
DROP TABLE IF EXISTS careplans;
DROP TABLE IF EXISTS conditions;
DROP TABLE IF EXISTS devices;
DROP TABLE IF EXISTS disease;
DROP TABLE IF EXISTS encounters;
DROP TABLE IF EXISTS imaging_studies;
DROP TABLE IF EXISTS immunizations;
DROP TABLE IF EXISTS patients;
DROP TABLE IF EXISTS procedures;
"""
conn.executescript(sql_commands)




# Commit the changes and close the database connection

In [44]:
conn.commit()
conn.close()